In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import gc
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from utils.schemas import *
from utils.functions import *

In [2]:
data_folder = 'input'

In [3]:
train = pd.read_csv(data_folder+'/train_ft_eng_0.csv', dtype = schema_generated_0)
test = pd.read_csv(data_folder+'/test_ft_eng_0.csv', dtype = schema_generated_0)

In [17]:
# mini_train = train.sample(100000, random_state = 42).fillna(X.median())

In [66]:
# X_cols = [x for x in train.columns if x not in ['isFraud', 'TransactionDT', 'TransactionID']]

In [109]:
X = train.sort_values('TransactionDT')[X_cols].fillna(X.median())
y = train.sort_values('TransactionDT').isFraud

In [110]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X.iloc[:int(X.shape[0]*0.8), :]
X_test = X.iloc[int(X.shape[0]*0.8):, :]
y_train = y[:int(X.shape[0]*0.8)]
y_test = y[int(X.shape[0]*0.8):]

In [111]:
# rf = RandomForestClassifier(n_estimators=500, oob_score=True, n_jobs=-1, random_state=42)
# rf.fit(X_train, y_train)

In [112]:
gc.collect()

14191

In [25]:
# perm = PermutationImportance(rf).fit(X_test, y_test)

In [26]:
# eli5.show_weights(perm)

In [27]:
import lightgbm as lgb
from sklearn.metrics import auc, precision_recall_curve

In [113]:
params = {
    'max_depth': -1,
    'num_leaves': 256,
    'metric': ['AUC'],
    'first_metric_only': True,
    'n_estimators': 20000,
    'learning_rate': 0.05,
    'colsample_bytree': 0.8,
    'objective': 'xentropy',
    'n_jobs': -1,
    'bagging_fraction': 0.8,
    'bagging_freq': 7,
    'lambda_l1': 0,
    'lambda_l2': 0,
    'bagging_seed': 42,
    'seed': 42,
    'feature_fraction_seed': 42,
    'drop_seed': 42,
    'data_random_seed': 42,
    'scale_pos_weight': 3,
}

In [114]:
def custom_loss(y_pred, y_true):
    precision, recall, thresholds = precision_recall_curve(np.where(y_true >= 0.5, 1, 0), y_pred)
    AUC = auc(recall, precision)
    if AUC != AUC:
        AUC = 0
    return 'PR_AUC', AUC, True

In [115]:
lgb_model = lgb.LGBMClassifier(**params)

In [116]:
y.value_counts()

0.0    569877
1.0     20663
Name: isFraud, dtype: int64

In [118]:
lgb_model.fit(X_train,
                   y_train,
                   eval_set=[(X_test, y_test)],
                   verbose=50,
                   early_stopping_rounds=50,
#                    eval_metric=custom_loss
                  )

Training until validation scores don't improve for 50 rounds.
[50]	valid_0's auc: 0.9037


KeyboardInterrupt: 

In [86]:
lgb_imp = lgb_model.feature_importances_/lgb_model.feature_importances_.max()

In [58]:
from catboost import CatBoostClassifier

In [60]:
cat_ft_id = list()
n = 0
for c in X_cols:
    if c in cat_ft:
        cat_ft_id.append(n)
    n += 1

In [70]:
params = {'depth':13,
          'iterations':20000,
          'eval_metric':'AUC',
          'random_seed':42,
          'logging_level':'Verbose',
          'allow_writing_files':False,
          'early_stopping_rounds':20,
          'learning_rate':0.07,
          'thread_count':8,
          'boosting_type':'Plain',
          'bootstrap_type':'Bernoulli',
          'rsm':0.3}

In [71]:
model_cb = CatBoostClassifier(**params)

In [72]:
model_cb.fit(X_train,
             y_train,
             cat_features=cat_ft_id,
             eval_set=(X_test, y_test),
             verbose=50
             )

0:	test: 0.6863300	best: 0.6863300 (0)	total: 1.98s	remaining: 32m 59s
10:	test: 0.8131691	best: 0.8131691 (10)	total: 10s	remaining: 15m 3s
20:	test: 0.8521409	best: 0.8521409 (20)	total: 23.8s	remaining: 18m 30s
30:	test: 0.8595719	best: 0.8595719 (30)	total: 39.6s	remaining: 20m 39s
40:	test: 0.8755269	best: 0.8755269 (40)	total: 59.4s	remaining: 23m 9s
50:	test: 0.8801712	best: 0.8801712 (50)	total: 1m 19s	remaining: 24m 44s
60:	test: 0.8834761	best: 0.8834761 (60)	total: 1m 38s	remaining: 25m 18s
70:	test: 0.8841216	best: 0.8845685 (64)	total: 1m 53s	remaining: 24m 48s
80:	test: 0.8856388	best: 0.8856388 (80)	total: 2m 13s	remaining: 25m 14s
90:	test: 0.8866321	best: 0.8866337 (89)	total: 2m 33s	remaining: 25m 31s
100:	test: 0.8870006	best: 0.8870006 (100)	total: 2m 48s	remaining: 24m 55s
110:	test: 0.8877822	best: 0.8877822 (110)	total: 3m 1s	remaining: 24m 17s
120:	test: 0.8890662	best: 0.8890662 (120)	total: 3m 18s	remaining: 24m 2s
130:	test: 0.8906125	best: 0.8906125 (130)	to

In [85]:
cb_imp = model_cb.feature_importances_/model_cb.feature_importances_.max()

In [89]:
model_rf = RandomForestClassifier(n_estimators=500, oob_score=True, n_jobs=-1, random_state=42)
model_rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=True, random_state=42, verbose=0,
                       warm_start=False)

In [91]:
rf_imp = model_rf.feature_importances_/model_rf.feature_importances_.max()

In [92]:
df_imp = pd.DataFrame({'feature': X_train.columns, 'importance': lgb_imp+cb_imp+rf_imp})\
.sort_values('importance', ascending = False)

In [108]:
df_imp.head(10)

,feature,importance
43,TransactionAmt,2.888210
552,N1,2.307518
385,card1,2.240519
4,C13,2.168373
0,C1,1.924876
554,N3,1.717469
386,card2,1.676416
551,Transaction_hour,1.652900
5,C14,1.449858
503,addr1_fe2,1.343054


In [94]:
import datetime

In [105]:
today = datetime.date.today()
D = today.strftime('%Y%m%d')

In [107]:
df_imp.to_csv('docs/ft_importances_{}.csv'.format(D), index=None, header=True)